<a href="https://colab.research.google.com/github/cogitare-aude/datasharing/blob/master/Copy_of_RepoGet_Github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#General Code

#give access to My Drive (Google Drive)
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#define path for this project
import os
os.chdir('drive/My Drive/Colab Notebooks/Capstone')

In [ ]:
#install library to manage Github via. Python
%pip install PyGithub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#Libraries

#for Git API
from github import Github 

#for API sleep timer
import calendar
import time
import datetime

#dealing with website headers
import json
import requests

#regex
import re

#for saving to disk
import pickle

#progress bar
from ipywidgets import IntProgress
from IPython.display import display

#for dynamic printing
#from sys import stdout

import pandas as pd


In [ ]:
#Github API 

#Github Auth Token (private)
with open('credentials.txt', 'rb') as file:
  lines = file.readlines()
  git_hub_cred = lines[0]
  git_hub_cred = git_hub_cred.decode('utf-8') #decode bytes

#Authenticate to create Github Object (API-auth token)
git_obj = Github(git_hub_cred)

In [ ]:
#Github API limit 
#ref.https://github.com/PyGithub/PyGithub/issues/1319

core_rate_limit = git_obj.get_rate_limit().core #24-hour 5000 pings to GitHub, returns reset-DTM
reset_timestamp = calendar.timegm(core_rate_limit.reset.timetuple()) #returns the corresponding Unix timestamp value
sleep_time = reset_timestamp - calendar.timegm(time.gmtime()) + 5  #add 5 seconds to be sure the rate limit has been reset
#system waits until (1-hr) re-set to continue
#time.sleep(sleep_time)

In [ ]:

#top 30 repos (defualt page size =30)
repos = results_dict['items']
len(repos)

repo_df = pd.DataFrame(repos)

In [ ]:
real_issues = [issue_content for i in range(round(issues.totalCount / 30) + 1) for issue_content in issues.get_page(i) if not issue_content.pull_request]
#issues-object includes git-issues & pull-requests, only keep real-issues
#ea. page (more issues = more pages) contains a list of issues, only keep issues that are real-git-issues (not PRs)


In [ ]:
rtn = {}
for issue_content in real_issues:
  iss_dict = {"title": issue_content.title, #title of issue
              "body": issue_content.body, #original comment
              "state": issue_content.state,
              "comments": [comment.body for comment in issue_content.get_comments()]} #follow-up comments
  rtn[issue_content.number] = iss_dict


#note:
# ```content``` -> quoted code chunck
# `conetent` -> quoted code line
# _italic_, **bold**, [hyperlink](www.w.com)
# ea. comment after body is separated by "" or ''

In [ ]:
real_issues[0]

Issue(title="Question Importing Data (P1)", number=3)

In [ ]:
for comment in real_issues[0].get_comments():
  print(comment)
  #comment.user
  #comment.created_at
  for reaction in comment.get_reactions():
    print(reaction.content)


IssueComment(user=NamedUser(login="JohannaPferd"), id=974707646)
+1
IssueComment(user=NamedUser(login="cogitare-aude"), id=1153131919)
heart
-1
IssueComment(user=NamedUser(login="cogitare-aude"), id=1153148525)


In [ ]:
reaction.content

'heart'

In [ ]:
#all parsable data from git-issue-comment object
comment.raw_data


In [ ]:
#all parsable data from git-issue object
real_issues[0].raw_data

In [ ]:
#ref. https://stackoverflow.com/questions/57211865/downloading-issues-from-a-public-github-repository

def download_github_issues_as_dict(repo_url, token):
    '''
    since GitHub doesn't make it super easy to download GitHub issues...
    :param repo_url: the full URL of the repo (don't include the trailing "/" nor ".git" suffix).
    :param token: a GitHub Personal Access Token (create from GitHub itself)
    :return: a dictionary that can be easily json-ified with the relevant info from the issues.
    '''

    assert isinstance(repo_url, str) and not repo_url.endswith("/") and repo_url.endswith(".git") and "/" in repo_url , "need proper url: 'https://github.com/repo-owner/repo-name'"
    assert isinstance(token, str), "need str-type unique Github token"

    git_obj = github.Github(token)
    user_str, repo_str = repo_url.replace("https://github.com/", "").split("/")
    user = git_obj.get_user(user_str)
    repo = user.get_repo(repo_str)
    issues = repo.get_issues(state="all")

    # (30 issues per page, find total number of pages that need parsing) the number 30 is hardcoded into the PyGitHub
    real_issues = [_ for i in range(round(issues.totalCount / 30) + 1) for _ in issues.get_page(i) if
                   not _.pull_request]
    rtn = {}
    for iss in real_issues:
        iss_dict = {"title": iss.title,
                    "body": iss.body,
                    "state": iss.state,
                    "comments": [_.body for _ in iss.get_comments()]}
        rtn[iss.number] = iss_dict

    return rtn

In [ ]:
#input url
#repo_url = 'https://github.com/freeCodeCamp/freeCodeCamp'
repo_url = 'https://github.com/lse-my470/assignment-8-centipede'

user_str, repo_str = repo_url.replace("https://github.com/", "").split("/")
user = git_obj.get_user(user_str)
repo = user.get_repo(repo_str)
issues = repo.get_issues(state="all")

#used 2 API

In [ ]:
#pd.DataFrame(repos)

,3,2
title,Question Importing Data (P1),Question Empty Vector
body,during which question /step should we remove t...,1. prof seems to always declare vectors with `...
create_dt,2021-11-20 19:06:30,2021-11-20 19:04:27
state,closed,closed
comments,[I agree that we shold remove the first 2 colu...,[I've tried both c(0) and c() in R.\r\nc(0) is...
comment_user,"[91499393, 35208282, 35208282]",[91499393]
comment_reaction,"[+1, +1, +1]",[+1]


In [ ]:
#input url
repo_url = 'https://github.com/freeCodeCamp/freeCodeCamp'
#repo_url = 'https://github.com/lse-my470/assignment-8-centipede'
#repo_url = 'https://github.com/borisdayma/dalle-mini'


user_str, repo_str = repo_url.replace("https://github.com/", "").split("/")
user = git_obj.get_user(user_str)
repo = user.get_repo(repo_str)
issues = repo.get_issues(state="all")


#####################################
###Step 1 Load All Issues
print('Step 1: Load all Issues')

#progress bar
#ref. https://stackoverflow.com/questions/38861829/how-do-i-implement-a-progress-bar
max_count = round(issues.totalCount / 30) #max loops (ie. pages) needed (starts at 0)
status_bar = IntProgress(min = 0, max = max_count) # create status-bar
display(status_bar)

#intialize: empty list
issue_content_list = []
total_page = 'out of ' + str(max_count) + ' pages' #for current page dynamic update
core_rate_limit = git_obj.get_rate_limit().core

#issues-obj contains all the issues + PR of a repo
#ea. page (more issues = more pages) contains a list of issues, only keep issues that are real-git-issues (not PRs)
for page_num in range(round(issues.totalCount / 30) + 1):
  #dynamically print current page_num & API limit
  print('page: ', f'\r{page_num, total_page, core_rate_limit}', end = '', flush = True)

  #API throttle
  try:

    #normal operations
    for issue_content in issues.get_page(page_num):
      if not issue_content.pull_request:
        issue_content_list.append(issue_content)

    #for status bar (before next page_num):
    status_bar.value += 1 #update status-bar
    #for calculating Github API limit:
    core_rate_limit = git_obj.get_rate_limit().core

    if core_rate_limit.remaining <= 50:
      reset_timestamp = calendar.timegm(core_rate_limit.reset.timetuple()) #returns the corresponding Unix timestamp value
      sleep_time = reset_timestamp - calendar.timegm(time.gmtime()) + 5 #add 5 sec (1-hr reset time calc. at start of execution)
      print('\nsleeping for:', round(sleep_time/60), 'mins;', ' at page: ', page_num, '###API hourly refresh limit: ', core_rate_limit)
      time.sleep(sleep_time)

  #after sleeping, continue back into above loop  
  except:
    continue

#save list to Disk (back-up)
with open('issue_content_list.pkl', 'wb') as file:
  pickle.dump(issue_content_list, file)

print('\n***issue_content_list: loading Repo Issues Complete!***')


Step 1: Load all Issues


IntProgress(value=0, max=0)

(0, 'out of 0 pages', Rate(reset=2022-06-14 20:31:56, remaining=4891, limit=5000))
***issue_content_list: loading Repo Issues Complete!***


In [ ]:
#load saved list for Step 2.
with open('issue_content_list.pkl', 'rb') as file:
    issue_content_list = pickle.load(file)

#for testing
#issue_content_list = issue_content_list[0:10]

In [ ]:
#testing

[]

In [ ]:
#Dynamically Saving

#####################################
###Step 2 Load Issue Contents to DICT
print('Step 2: Load Issues to DICT')

#progress bar
max_count = len(issue_content_list) #max loops (ie. isuues) needed
status_bar_ii = IntProgress(min = 1, max = max_count) # create status-bar (starts at 1)
display(status_bar_ii)

#initialize: empty dict
repo_dict = {}
total_count = 'out of ' + str(max_count) + ' issues' #for current issue dynamic update
core_rate_limit = git_obj.get_rate_limit().core

#initialize counter of ea. Batche-file & Issue-tracker
#only load the issues not processed in previous batch
try:
  #if start already exist (ie. batch-processing commenced or single-completion)
  with open('issue_tracker.pkl', 'rb') as file:
    issue_start_num = pickle.load(file) 
    issue_content_list = issue_content_list[issue_start_num: ]
  if issue_start_num == 0: #re-starting
    batch_count = -1
except FileNotFoundError:
  start = status_bar_ii.value - 1 #selecting the issue_num where the last batch stoped (starts @ 0)
  batch_count = -1 #initialize counter of ea. batche-file
  issue_content_list = issue_content_list[start: ]

#Start of actual issue processing
for issue_content in issue_content_list:
  #dynamically print current issue & API limit
  print('process issue: ', f'\r{status_bar_ii.value, total_count, core_rate_limit}', end = '', flush = True)

  #API Throttle
  try:
    issue_dict = {'title': issue_content.title, #title of issue
                  'body': issue_content.body, #original comment
                  'created_at': issue_content.created_at, #dtm of issue
                  'state': issue_content.state,
                  'comments': [comment.body for comment in issue_content.get_comments()], #follow-up comments
                  'comment_user': [comment.user.id for comment in issue_content.get_comments()], #follow-up commentor user-ID
                  #reactions for ea. comment in ea. issue
                  'comment_react': [ 
                                    [reaction.content for reaction in comment.get_reactions()] #nested loop
                                    for comment in issue_content.get_comments()]
                  }
                  
    #add to dict: ea. Issue (many comments) belongs to one key in dict
    repo_dict[issue_content.number] = issue_dict

    #for status bar:
    status_bar_ii.value += 1 #update status-bar
    start += 1 #update issue processed

    #for calculating Github API limit:
    core_rate_limit = git_obj.get_rate_limit().core
    
    if core_rate_limit.remaining <= 50:

      #calc. sleep timer 
      reset_timestamp = calendar.timegm(core_rate_limit.reset.timetuple()) #returns the corresponding Unix timestamp value
      sleep_time = reset_timestamp - calendar.timegm(time.gmtime()) + 5 #add 5 sec (1-hr reset time calc. at start of execution)
     
      #save batch of processed issues into batch-file
      batch_count += 1 #for each batch-dict
      file_batch = 'repo_dict_batch_' + str(batch_count) + '.pkl'

      with open(file_batch, 'wb') as file:
        pickle.dump(repo_dict, file)

      #record batch saving progress
      with open('issue_tracker.pkl', 'wb') as file:
        pickle.dump(start, file)
  
      repo_dict = {} #refresh temp-dict after batch saved

      #sleep
      print('\nbatch saved & sleeping for:', round(sleep_time/60), 'mins;', ' at issue:', status_bar_ii.value, '###API hourly refresh limit: ', core_rate_limit)
      time.sleep(sleep_time) 

  #after sleeping, continue back into above loop  
  except:
    continue

#IF no sleeping required: 
if batch_count == -1:
  #saving DICT to file in one go
  with open('repo_dict.pkl', 'wb') as file:
    pickle.dump(repo_dict, file)
  #updating issues processed (tracker)
  with open('issue_tracker.pkl', 'wb') as file:
        pickle.dump(start, file)

#All Issues finished processing
batch_count = -1 #reset (start from 0)
start = 0 #reset
print('\n***repo_dict: processing Issues into DICT complete!***', end = '\n')

#remember to reset the issue tracker!

Step 2: Load Issues to DICT


IntProgress(value=1, max=16065, min=1)

(914, 'out of 16065 issues', Rate(reset=2022-06-15 22:57:03, remaining=55, limit=5000))
batch saved & sleeping for: 52 mins;  at issue: 915 ###API hourly refresh limit:  Rate(reset=2022-06-15 22:57:03, remaining=45, limit=5000)
(915, 'out of 16065 issues', Rate(reset=2022-06-15 22:57:03, remaining=45, limit=5000))
batch saved & sleeping for: 52 mins;  at issue: 916 ###API hourly refresh limit:  Rate(reset=2022-06-15 22:57:03, remaining=32, limit=5000)
(916, 'out of 16065 issues', Rate(reset=2022-06-15 22:57:03, remaining=32, limit=5000))
batch saved & sleeping for: 52 mins;  at issue: 917 ###API hourly refresh limit:  Rate(reset=2022-06-15 22:57:03, remaining=22, limit=5000)
(917, 'out of 16065 issues', Rate(reset=2022-06-15 22:57:03, remaining=22, limit=5000))
batch saved & sleeping for: 51 mins;  at issue: 918 ###API hourly refresh limit:  Rate(reset=2022-06-15 22:57:03, remaining=18, limit=5000)
(918, 'out of 16065 issues', Rate(reset=2022-06-15 22:57:03, remaining=18, limit=5000))


In [ ]:
#reset issue tracker
start = 0
with open('issue_tracker.pkl', 'wb') as file:
  pickle.dump(start, file)

In [ ]:
with open('issue_tracker.pkl', 'rb') as file:
  sample = pickle.load(file)

sample

FileNotFoundError: ignored

In [ ]:
with open('repo_dict_batch_1.pkl', 'rb') as file:
  load_file = pickle.load(file)

load_file

{46481: {'body': '### Describe the Issue\n\nUnclear Instructions in step 41 of the Learn CSS Variables by Building a City Skyline of the (New) Responsive Web Design course.\r\n\r\nYou want to add the same gradient to the next two sections. Instead of doing that, **create a new class called bb1-window,** and move the height and background properties and values from .bb1a **to the new class**.\r\n\r\nI believe telling the user to create a new class **selector** instead would make the instructions more clear.\r\n\r\n\n\n### Affected Page\n\nhttps://www.freecodecamp.org/learn/2022/responsive-web-design/learn-css-variables-by-building-a-city-skyline/step-41\n\n### Your code\n\n```\r\n.bb1-window {\r\n    height: 10%;\r\n    background: linear-gradient(\r\n      var(--building-color1),\r\n      var(--window-color1)\r\n      );\r\n}\r\n```\r\n\n\n### Expected behavior\n\nInstructions should be given as:\r\n\r\nYou want to add the same gradient to the next two sections. Instead of doing that, 

# New Section

In [ ]:
file_batch = 'issue_dict_batch_' + str(batch_count) + '.pkl'
file_batch

'issue_dict_batch_10.pkl'

In [ ]:
with open('repo_dict.pkl', 'rb') as file:
  load_file = pickle.load(file)

load_file

EOFError: ignored

In [ ]:
%time
#dir(comment) #all attributes for the obj.

issue_content.get_comments()

In [ ]:
        
with open('repo_dict.pkl', 'rb') as f:
    loaded_dict = pickle.load(f)

loaded_dict

{}

['+1']

## Star History

In [ ]:
#using Github Repo Search API
#ref. https://towardsdatascience.com/top-30-github-python-projects-at-the-beginning-of-2022-86b1e3dad1a

url = 'https://api.github.com/search/repositories?q=language:python&sort=stars&order=desc'

#use request module to get url's data
results = requests.get(url)
#conver to python dictionary
results_dict = results.json()

#top 30 repos (defualt page size =30)
repos = results_dict['items']
len(repos)

repo_df = pd.DataFrame(repos)


repo_df_clean = repo_df[['name', 'full_name', 'html_url', 'created_at', 'stargazers_count', 'watchers', 'forks', 'open_issues']]
repo_df_clean['created_at'] = pd.to_datetime(repo_df_clean['created_at'])
repo_df_clean['created_year'] = repo_df_clean['created_at'].dt.year
repo_df_clean['years_on_github'] = 2022 - repo_df_clean['created_at'].dt.year

repo_df_clean.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

,name,full_name,html_url,created_at,stargazers_count,watchers,forks,open_issues,created_year,years_on_github
0,public-apis,public-apis/public-apis,https://github.com/public-apis/public-apis,2016-03-20 23:49:42+00:00,196118,196118,22598,2,2016,6
1,system-design-primer,donnemartin/system-design-primer,https://github.com/donnemartin/system-design-p...,2017-02-26 16:15:28+00:00,183676,183676,33436,313,2017,5
2,awesome-python,vinta/awesome-python,https://github.com/vinta/awesome-python,2014-06-27 21:00:06+00:00,130014,130014,21281,252,2014,8
3,Python-100-Days,jackfrued/Python-100-Days,https://github.com/jackfrued/Python-100-Days,2018-03-01 16:05:52+00:00,119772,119772,45776,643,2018,4
4,youtube-dl,ytdl-org/youtube-dl,https://github.com/ytdl-org/youtube-dl,2010-10-31 14:35:07+00:00,110725,110725,7842,4727,2010,12


In [ ]:
pd.set_option('max_colwidth', None)
pd.set_option('max_colwidth', None)
pd.set_option('display.max_columns', None)
#reset 
#pd.reset_option(“max_columns”)
repo_df.loc[repo_df['id'] == 123458551]


,id,node_id,name,full_name,private,owner,html_url,description,fork,url,forks_url,keys_url,collaborators_url,teams_url,hooks_url,issue_events_url,events_url,assignees_url,branches_url,tags_url,blobs_url,git_tags_url,git_refs_url,trees_url,statuses_url,languages_url,stargazers_url,contributors_url,subscribers_url,subscription_url,commits_url,git_commits_url,comments_url,issue_comment_url,contents_url,compare_url,merges_url,archive_url,downloads_url,issues_url,pulls_url,milestones_url,notifications_url,labels_url,releases_url,deployments_url,created_at,updated_at,pushed_at,git_url,ssh_url,clone_url,svn_url,homepage,size,stargazers_count,watchers_count,language,has_issues,has_projects,has_downloads,has_wiki,has_pages,forks_count,mirror_url,archived,disabled,open_issues_count,license,allow_forking,is_template,topics,visibility,forks,open_issues,watchers,default_branch,score
0,123458551,MDEwOlJlcG9zaXRvcnkxMjM0NTg1NTE=,Python-100-Days,jackfrued/Python-100-Days,False,"{'login': 'jackfrued', 'id': 7474657, 'node_id': 'MDQ6VXNlcjc0NzQ2NTc=', 'avatar_url': 'https://avatars.githubusercontent.com/u/7474657?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/jackfrued', 'html_url': 'https://github.com/jackfrued', 'followers_url': 'https://api.github.com/users/jackfrued/followers', 'following_url': 'https://api.github.com/users/jackfrued/following{/other_user}', 'gists_url': 'https://api.github.com/users/jackfrued/gists{/gist_id}', 'starred_url': 'https://api.github.com/users/jackfrued/starred{/owner}{/repo}', 'subscriptions_url': 'https://api.github.com/users/jackfrued/subscriptions', 'organizations_url': 'https://api.github.com/users/jackfrued/orgs', 'repos_url': 'https://api.github.com/users/jackfrued/repos', 'events_url': 'https://api.github.com/users/jackfrued/events{/privacy}', 'received_events_url': 'https://api.github.com/users/jackfrued/received_events', 'type': 'User', 'site_admin': False}",https://github.com/jackfrued/Python-100-Days,Python - 100天从新手到大师,False,https://api.github.com/repos/jackfrued/Python-100-Days,https://api.github.com/repos/jackfrued/Python-100-Days/forks,https://api.github.com/repos/jackfrued/Python-100-Days/keys{/key_id},https://api.github.com/repos/jackfrued/Python-100-Days/collaborators{/collaborator},https://api.github.com/repos/jackfrued/Python-100-Days/teams,https://api.github.com/repos/jackfrued/Python-100-Days/hooks,https://api.github.com/repos/jackfrued/Python-100-Days/issues/events{/number},https://api.github.com/repos/jackfrued/Python-100-Days/events,https://api.github.com/repos/jackfrued/Python-100-Days/assignees{/user},https://api.github.com/repos/jackfrued/Python-100-Days/branches{/branch},https://api.github.com/repos/jackfrued/Python-100-Days/tags,https://api.github.com/repos/jackfrued/Python-100-Days/git/blobs{/sha},https://api.github.com/repos/jackfrued/Python-100-Days/git/tags{/sha},https://api.github.com/repos/jackfrued/Python-100-Days/git/refs{/sha},https://api.github.com/repos/jackfrued/Python-100-Days/git/trees{/sha},https://api.github.com/repos/jackfrued/Python-100-Days/statuses/{sha},https://api.github.com/repos/jackfrued/Python-100-Days/languages,https://api.github.com/repos/jackfrued/Python-100-Days/stargazers,https://api.github.com/repos/jackfrued/Python-100-Days/contributors,https://api.github.com/repos/jackfrued/Python-100-Days/subscribers,https://api.github.com/repos/jackfrued/Python-100-Days/subscription,https://api.github.com/repos/jackfrued/Python-100-Days/commits{/sha},https://api.github.com/repos/jackfrued/Python-100-Days/git/commits{/sha},https://api.github.com/repos/jackfrued/Python-100-Days/comments{/number},https://api.github.com/repos/jackfrued/Python-100-Days/issues/comments{/number},https://api.github.com/repos/jackfrued/Python-100-Days/contents/{+path},https://api.github.com/repos/jackfrued/Python-100-Days/compare/{base}...{head},https://api.github.com/repos/jackfrued/Python-100-Days/merges,https://api.github.com/repos/jackfrued/Python-100-Days/{archive_forma

In [ ]:
repo_df.columns

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement star-history (from versions: none)
ERROR: No matching distribution found for star-history


In [ ]:
#stars overtime

url = 'https://api.github.com/repos/freeCodeCamp/freeCodeCamp/stargazers'


#use request module to 
repo_response = requests.get(url,headers={'Accept': 'application/vnd.github.v3.star+json'})
#stars = results.headers

print(repo_response.json())



[{'starred_at': '2014-12-24T20:05:35Z', 'user': {'login': 'BerkeleyTrue', 'id': 6775919, 'node_id': 'MDQ6VXNlcjY3NzU5MTk=', 'avatar_url': 'https://avatars.githubusercontent.com/u/6775919?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/BerkeleyTrue', 'html_url': 'https://github.com/BerkeleyTrue', 'followers_url': 'https://api.github.com/users/BerkeleyTrue/followers', 'following_url': 'https://api.github.com/users/BerkeleyTrue/following{/other_user}', 'gists_url': 'https://api.github.com/users/BerkeleyTrue/gists{/gist_id}', 'starred_url': 'https://api.github.com/users/BerkeleyTrue/starred{/owner}{/repo}', 'subscriptions_url': 'https://api.github.com/users/BerkeleyTrue/subscriptions', 'organizations_url': 'https://api.github.com/users/BerkeleyTrue/orgs', 'repos_url': 'https://api.github.com/users/BerkeleyTrue/repos', 'events_url': 'https://api.github.com/users/BerkeleyTrue/events{/privacy}', 'received_events_url': 'https://api.github.com/users/BerkeleyTrue/received_events', 

## Contributors

In [ ]:

user_str, repo_str = 'https://github.com/freeCodeCamp/freeCodeCamp'.replace("https://github.com/", "").split("/")

user = git_obj.get_user(user_str)
repo = user.get_repo(repo_str)


#total contributors
repo.get_contributors().totalCount

406

In [ ]:
url = 'https://api.github.com/repos/freeCodeCamp/freeCodeCamp/contributors?per_page=1&anon=true'


#use request module to get url's data
results = requests.get(url)

test = results.headers


In [ ]:
dict(test)['Link']

'<https://api.github.com/repositories/28457823/contributors?per_page=1&anon=true&page=2>; rel="next", <https://api.github.com/repositories/28457823/contributors?per_page=1&anon=true&page=4884>; rel="last"'

In [ ]:
repo_df = pd.DataFrame(repos)

In [ ]:
#convert to json string
test_dict = json.dumps(dict(test))

test_dict

'{"Server": "GitHub.com", "Date": "Sun, 12 Jun 2022 19:58:37 GMT", "Content-Type": "application/json; charset=utf-8", "Cache-Control": "public, max-age=60, s-maxage=60", "Vary": "Accept, Accept-Encoding, Accept, X-Requested-With", "ETag": "W/\\"e5f9c870f98f312133f917228ddd147bcec8a21f96b96762a18aa5f5a463eeea\\"", "Last-Modified": "Sun, 12 Jun 2022 19:37:37 GMT", "X-GitHub-Media-Type": "github.v3; format=json", "Link": "<https://api.github.com/repositories/28457823/contributors?per_page=1&anon=true&page=2>; rel=\\"next\\", <https://api.github.com/repositories/28457823/contributors?per_page=1&anon=true&page=4884>; rel=\\"last\\"", "Access-Control-Expose-Headers": "ETag, Link, Location, Retry-After, X-GitHub-OTP, X-RateLimit-Limit, X-RateLimit-Remaining, X-RateLimit-Used, X-RateLimit-Resource, X-RateLimit-Reset, X-OAuth-Scopes, X-Accepted-OAuth-Scopes, X-Poll-Interval, X-GitHub-Media-Type, X-GitHub-SSO, X-GitHub-Request-Id, Deprecation, Sunset", "Access-Control-Allow-Origin": "*", "Strict

In [ ]:
x = re.findall('true&page=(\d+)', test_dict)

x[1]

'4884'

## Apendix Code

In [ ]:
#####################################
###Step 2 Load Issue Contents to DICT
print('Step 2: Load Issues to DICT')

#progress bar
max_count = len(issue_content_list) #max loops (ie. isuues) needed
status_bar_ii = IntProgress(min = 1, max = max_count) # create status-bar (starts at 1)
display(status_bar_ii)

#intialize: empty dict
repo_dict = {}
total_count = 'out of ' + str(max_count) + ' issues' #for current issue dynamic update
core_rate_limit = git_obj.get_rate_limit().core

for issue_content in issue_content_list:
  #dynamically print current issue & API limit
  print('process issue: ', f'\r{status_bar_ii.value, total_count, core_rate_limit}', end = '', flush = True)

  #API Throttle
  try:
    issue_dict = {'title': issue_content.title, #title of issue
                  'body': issue_content.body, #original comment
                  'created_at': issue_content.created_at, #dtm of issue
                  'state': issue_content.state,
                  'comments': [comment.body for comment in issue_content.get_comments()], #follow-up comments
                  'comment_user': [comment.user.id for comment in issue_content.get_comments()], #follow-up commentor user-ID
                  #reactions for ea. comment in ea. issue
                  'comment_react': [ 
                                    [reaction.content for reaction in comment.get_reactions()] #nested loop
                                    for comment in issue_content.get_comments()]
                  }
                  
    #add to dict: ea. Issue (many comments) belongs to one key in dict
    repo_dict[issue_content.number] = issue_dict

    #for status bar:
    status_bar_ii.value += 1 #update status-bar
    #for calculating Github API limit:
    core_rate_limit = git_obj.get_rate_limit().core
    
    if core_rate_limit.remaining <= 50:
      reset_timestamp = calendar.timegm(core_rate_limit.reset.timetuple()) #returns the corresponding Unix timestamp value
      sleep_time = reset_timestamp - calendar.timegm(time.gmtime()) + 5 #add 5 sec (1-hr reset time calc. at start of execution)
      print('\nsleeping for:', sleep_time/60, 'mins;', ' at issue:', status_bar_ii.value, '###API hourly refresh limit: ', core_rate_limit)
      time.sleep(sleep_time) 

  #after sleeping, continue back into above loop  
  except:
    continue

#saving DICT to file
with open('repo_dict.pkl', 'wb') as f:
    pickle.dump(repo_dict, f)

print('\n***repo_dict: processing Issues into DICT complete!***', end = '\n')

In [ ]:
#explore other code

from github import Github
# using username and password
# or using an access token
g = Github("***************************")
for repo in g.get_user().get_repos():
    print(repo.name)

print("**********Get Current Repos**********")
user = g.get_user()
user.login
print(user.login)
repo = g.get_repo("<any-repo>/<any-repo>")
repo.name
print(repo.name)
print("********Get the Repo Topics**************")

repo = g.get_repo("<any-repo>/<any-repo>")
repo.get_topics()
print(repo.get_topics())

print("*****Get the Star Count*************")
repo = g.get_repo("<any-repo>/<any-repo>")
repo.stargazers_count
print(repo.stargazers_count)
print("********Get the Open Issues*********")
repo = g.get_repo("<any-repo>/<any-repo>")
open_issues = repo.get_issues(state='open')
for issue in open_issues:
    print(issue)

print("******Get the Branch Count*******")
repo = g.get_repo("<any-repo>/<any-repo>")
print(list(repo.get_branches()))

NameError: ignored

In [ ]:
#input url
#repo_url = 'https://github.com/freeCodeCamp/freeCodeCamp'
#repo_url = 'https://github.com/lse-my470/assignment-8-centipede'
repo_url = 'https://github.com/borisdayma/dalle-mini'


user_str, repo_str = repo_url.replace("https://github.com/", "").split("/")
user = git_obj.get_user(user_str)
repo = user.get_repo(repo_str)
issues = repo.get_issues(state="all")


#####################################
###Step 1 Load All Issues
print('Step 1: Load all Issues')

#progress bar
#ref. https://stackoverflow.com/questions/38861829/how-do-i-implement-a-progress-bar
max_count = round(issues.totalCount / 30) #max loops (ie. pages) needed
status_bar = IntProgress(min = 0, max = max_count) # create status-bar
display(status_bar)

#intialize: empty list
issue_content_list = []
total_page = 'out of ' + str(max_count) + ' pages' #for current page dynamic update
core_rate_limit = git_obj.get_rate_limit().core

#issues-obj contains all the issues + PR of a repo
#ea. page (more issues = more pages) contains a list of issues, only keep issues that are real-git-issues (not PRs)
for page_num in range(round(issues.totalCount / 30) + 1):
  #print('page: ', page_num, 'out of', max_count, 'API limit: ', core_rate_limit)
  #dynamically print current page_num & API limit
  print('page: ', f'\r{page_num, total_page, core_rate_limit}', end = '', flush = True)

  #API throttle
  try:

    #normal operations
    for issue_content in issues.get_page(page_num):
      if not issue_content.pull_request:
        issue_content_list.append(issue_content)

    #for status bar (before next page_num):
    status_bar.value += 1 #update status-bar
    #for calculating Github API limit:
    core_rate_limit = git_obj.get_rate_limit().core

    if core_rate_limit.remaining <= 100:
      reset_timestamp = calendar.timegm(core_rate_limit.reset.timetuple()) #returns the corresponding Unix timestamp value
      sleep_time = reset_timestamp - calendar.timegm(time.gmtime()) + 5 #add 5 seconds to be sure the rate limit has been reset
      print('sleeping for 1-hr at page: ', page_num, 'API limit: ', core_rate_limit)
      time.sleep(sleep_time)

  #after sleeping, continue back into above loop  
  except:
    continue

#####################################
print('issue_content_list: loading Repo Issues Complete!')


#####################################
###Step 2 Load Issue Contents to DICT
print('Step 2: Load Issues to DICT')

#progress bar
max_count = len(issue_content_list) #max loops (ie. pages) needed
status_bar = IntProgress(min = 0, max = max_count) # create status-bar
display(status_bar)

#intialize: empty dict
rtn = {}
total_page = 'out of ' + str(max_count) + ' pages' #for current page dynamic update
core_rate_limit = git_obj.get_rate_limit().core

for issue_content in issue_content_list:
  iss_dict = {"title": issue_content.title, #title of issue
              "body": issue_content.body, #original comment
              "create_dt": issue_content.created_at, #dtm of issue
              "state": issue_content.state,
              "comments": [comment.body for comment in issue_content.get_comments()], #follow-up comments
              "comment_user": [comment.user.id for comment in issue_content.get_comments()], #follow-up commentor user-ID
              "comment_reaction": [reaction.content for reaction in comment.get_reactions() for comment in issue_content.get_comments()] #follow-up commentor user-ID
              }
  #each Issue (many comments) belongs to one key in dictionary
  rtn[issue_content.number] = iss_dict


#progress bar
#ref. https://stackoverflow.com/questions/38861829/how-do-i-implement-a-progress-bar
max_count = round(issues.totalCount / 30) #max loops (ie. pages) needed
status_bar = IntProgress(min = 0, max = max_count) # create status-bar
display(status_bar)